In [1]:
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "path-to-your-json"

from google.cloud import bigquery
bigquery_client = bigquery.Client()

import pandas as pd

In [2]:
q1 = """
SELECT * 
FROM `learningbigquery-325010.OI.Olympic_Game`
  """
qr1 = bigquery_client.query(q1)
oi_data = qr1.to_dataframe()

oi_data.head()

,Nation,Abbreviation,Gold,Silver,Bronze,Total,Year,Type,Host
0,United States,USA,39,41,33,113,2020,Summer,Japan
1,People's Republic of China,CHN,38,32,18,88,2020,Summer,Japan
2,Japan,JPN,27,14,17,58,2020,Summer,Japan
3,Great Britain,GBR,22,21,22,65,2020,Summer,Japan
4,Russian Olympic Committee,ROC,20,28,23,71,2020,Summer,Japan


In [3]:
total_w = 3 * oi_data.Gold + 2 * oi_data.Silver + 1 * oi_data.Bronze
oi_data['Total_Weighted_Medals'] = total_w
oi_data

,Nation,Abbreviation,Gold,Silver,Bronze,Total,Year,Type,Host,Total_Weighted_Medals
0,United States,USA,39,41,33,113,2020,Summer,Japan,232
1,People's Republic of China,CHN,38,32,18,88,2020,Summer,Japan,196
2,Japan,JPN,27,14,17,58,2020,Summer,Japan,126
3,Great Britain,GBR,22,21,22,65,2020,Summer,Japan,130
4,Russian Olympic Committee,ROC,20,28,23,71,2020,Summer,Japan,139
...,...,...,...,...,...,...,...,...,...,...
842,Luxembourg,LUX,0,2,0,2,1992,Winter,France,4
843,New Zealand,NZL,0,1,0,1,1992,Winter,France,2
844,Czechoslovakia,TCH,0,0,3,3,1992,Winter,France,3
845,Democratic People's Republic of Korea,PRK,0,0,1,1,1992,Winter,France,1


In [4]:
oi_data = oi_data.groupby('Nation')['Total_Weighted_Medals'].sum().reset_index()
oi_data = oi_data.sort_values(by = 'Total_Weighted_Medals', ascending = False)

In [5]:
oi_data.head(10)

,Nation,Total_Weighted_Medals
128,United States,2146
88,People's Republic of China,1354
42,Germany,1285
98,Russian Federation,1100
44,Great Britain,719
39,France,706
4,Australia,661
57,Italy,638
95,Republic of Korea,597
18,Canada,590


In [6]:
q2 = """
SELECT Country__or_dependency_, Population__2020_ FROM `learningbigquery-325010.OI.Population_Countries_2020` 
ORDER BY Population__2020_ DESC
"""

qr2 = bigquery_client.query(q2)
pop = qr2.to_dataframe()

pop.head()

,Country__or_dependency_,Population__2020_
0,China,1438207241
1,India,1377233523
2,United States,330610570
3,Indonesia,272931713
4,Pakistan,219992900


In [7]:
pop = pop.rename(columns = {'Country__or_dependency_':'Nation',
                            'Population__2020_':'Population'})

pop.head()

,Nation,Population
0,China,1438207241
1,India,1377233523
2,United States,330610570
3,Indonesia,272931713
4,Pakistan,219992900


In [8]:
med_pop = pd.merge(oi_data,pop,on='Nation',how='inner')

med_pop

,Nation,Total_Weighted_Medals,Population
0,United States,2146,330610570
1,Germany,1285,83730223
2,France,706,65244628
3,Australia,661,25439164
4,Italy,638,60479424
...,...,...,...
111,Pakistan,1,219992900
112,Liechtenstein,1,38106
113,Barbados,1,287305
114,Mauritius,1,1271347


In [9]:
no_match = oi_data.loc[~oi_data['Nation'].isin(med_pop.Nation)]
no_match

,Nation,Total_Weighted_Medals
88,People's Republic of China,1354
98,Russian Federation,1100
44,Great Britain,719
95,Republic of Korea,597
126,Unified Team,287
26,Czech Republic,186
99,Russian Olympic Committee,166
55,Islamic Republic of Iran,98
29,Democratic People's Republic of Korea,82
20,Chinese Taipei,57


In [10]:
oi_data['Nation'].replace(
    
    ["People's Republic of China",
    "Russian Federation",
    "Great Britain",
    "Republic of Korea",
    "Czech Republic",
    "Russian Olympic Committee",
    "Islamic Republic of Iran",
    "Democratic People's Republic of Korea",
    "Chinese Taipei",
    "The Bahamas",
    "Hong Kong, China",
    "Serbia and Montenegro",
    "Republic of Moldova",
    "Kingdom of Saudi Arabia",
    "Syrian Arab Republic"],

    ["China",
     "Russia",
     "United Kingdom",
     "South Korea",
     "Czech Republic (Czechia)",
     "Russia",
     "Iran",
     "North Korea",
     "Taiwan",
     "Bahamas",
     "Hong Kong",
     "Serbia",
     "Moldova",
     "Saudi Arabia",
     "Syria"], inplace = True
)

oi_data

,Nation,Total_Weighted_Medals
128,United States,2146
88,China,1354
42,Germany,1285
98,Russia,1100
44,United Kingdom,719
...,...,...
85,Pakistan,1
68,Liechtenstein,1
8,Barbados,1
72,Mauritius,1


In [11]:
med_pop_con = pd.merge(oi_data,pop,on='Nation',how='inner')

med_pop_con

,Nation,Total_Weighted_Medals,Population
0,United States,2146,330610570
1,China,1354,1438207241
2,Germany,1285,83730223
3,Russia,1100,145922010
4,Russia,166,145922010
...,...,...,...
126,Pakistan,1,219992900
127,Liechtenstein,1,38106
128,Barbados,1,287305
129,Mauritius,1,1271347


In [12]:
med_w_pop = (med_pop_con.Total_Weighted_Medals / med_pop_con.Population)*100000
med_pop_con['Number Of Medals per 100k Inhabitants'] = med_w_pop
med_pop_con

,Nation,Total_Weighted_Medals,Population,Number Of Medals per 100k Inhabitants
0,United States,2146,330610570,0.649102
1,China,1354,1438207241,0.094145
2,Germany,1285,83730223,1.534691
3,Russia,1100,145922010,0.753827
4,Russia,166,145922010,0.113759
...,...,...,...,...
126,Pakistan,1,219992900,0.000455
127,Liechtenstein,1,38106,2.624259
128,Barbados,1,287305,0.348062
129,Mauritius,1,1271347,0.078657


In [13]:
med_pop_most_s = med_pop_con.sort_values(by = 'Number Of Medals per 100k Inhabitants', ascending = False)
med_pop_most_s.head(50)

,Nation,Total_Weighted_Medals,Population,Number Of Medals per 100k Inhabitants
99,San Marino,4,33917,11.793496
12,Norway,542,5412632,10.013613
60,Bahamas,30,392477,7.643760
90,Grenada,6,112418,5.337224
102,Bermuda,3,62323,4.813632
30,Jamaica,137,2958567,4.630620
40,Slovenia,79,2078881,3.800121
26,New Zealand,172,4814272,3.572710
16,Austria,295,8996022,3.279227
15,Hungary,316,9665192,3.269464


In [14]:
import atoti as tt

session = tt.create_session()

Welcome to atoti 0.6.4!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, set the ATOTI_DISABLE_TELEMETRY environment variable to True.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


In [15]:
med_pop_graph = session.read_pandas(med_pop_most_s, keys=["Nation"], table_name="Nation Medals")

In [16]:
med_pop_graph.head()

,Total_Weighted_Medals,Population,Number Of Medals per 100k Inhabitants
Nation,,,
San Marino,4,33917,11.793496
Norway,542,5412632,10.013613
Bahamas,30,392477,7.643760
Grenada,6,112418,5.337224
Bermuda,3,62323,4.813632


In [17]:
cube = session.create_cube(med_pop_graph)

In [18]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [19]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [20]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [21]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.